In [1]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook.  Specifically, this notebook shows:

1.   Exporting training/testing data from Earth Engine in TFRecord format.
2.   Preparing the data for use in a TensorFlow model.
2.   Training and validating a simple model (Keras `Sequential` neural network) in TensorFlow.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Ingesting classified image data to Earth Engine in TFRecord format.

# Install the Earth Engine client library

This only needs to be done once per notebook.

In [3]:
# pip install earthengine-api

# Authentication

To read/write from a Google Cloud Storage bucket to which you have access, it's necessary to authenticate (as yourself).  You'll also need to authenticate as yourself with Earth Engine, so that you'll have access to your scripts, assets, etc.

## Authenticate to Googl Earth Engine

Identify yourself to Google Cloud, so you have access to storage and other resources.  When you run the code below, it will display a link in the output to an authentication page in your browser.  Follow the link to a page that will let you grant permission to the Cloud SDK to access your resources.  Copy the code from the permissions page back into this notebook and press return to complete the process.

(You may need to run this again if you get a credentials error later.)

In [9]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

In [10]:
import folium
print(folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

0.10.1


In [13]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# # Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = l8sr.filterDate('2018-01-01', '2018-02-28').map(maskL8sr).median()



In [14]:
# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[-35.7, 144.35])
folium.TileLayer(
    tiles=EE_TILES.format(**mapid),
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map